In [2]:
from yahoo_finance import Share
import requests
import base64
from bs4 import BeautifulSoup
import pandas
import sqlite3 as lite
from shutil import copyfileobj
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from datetime import datetime , date 
import sys
import os
import re
from time import sleep
import mysql.connector

#define max stock id
max_stock_id = 1000

#payload data for twse
payload={}
dt_str = []
#gmail setting
emailfrom = "gghammer@gmail.com"
emailto = "gghammer@gmail.com"
#fileToSend = "daily.xlsx"
username = "gghammer"
password = "ggpudin81161"

def conn_mysql():
    try:
        server = '192.168.0.110'    
        cnx = mysql.connector.connect(user='stock', password='stock168',host= server,database='stock')  
        print("Connector to MYSQL Server:",server,"SUCCESS")
        return cnx 
    except:   
        try:
            server = 'ggnas.synology.me'
            cnx = mysql.connector.connect(user='stock', password='stock168',host= server,database='stock')   
            print("Connector to MYSQL Server:",server,"SUCCESS")
            return cnx 
        except:
            print("Connector to MYSQL Server FAIL") 
    
def send_to_gmail(source,attach_name):    
    #save to xlsx file
    twse = pandas.read_csv(source,encoding = "big5",header=1)    
    writer = pandas.ExcelWriter(attach_name, engine='xlsxwriter')
    twse.to_excel(writer, sheet_name='Sheet1')
    writer.save()

    #gmail send 
    msg = MIMEMultipart()
    msg["From"] = emailfrom
    msg["To"] = emailto
    today = date.today()    
    #msg["Subject"] = "哈墨投顧日報 "+currenttime.strftime('%Y-%m-%d')
    msg["Subject"] = "哈墨投顧日報 "+str(today)    
    msg.preamble = "help I cannot send an attachment to save my life"

    ctype, encoding = mimetypes.guess_type(attach_name)
    if ctype is None or encoding is not None:
        ctype = "application/octet-stream"

    maintype, subtype = ctype.split("/", 1)

    if maintype == "text":
        fp = open(attach_name)
        # Note: we should handle calculating the charset
        attachment = MIMEText(fp.read(), _subtype=subtype)
        fp.close()
    elif maintype == "image":
        fp = open(attach_name, "rb")
        attachment = MIMEImage(fp.read(), _subtype=subtype)
        fp.close()
    elif maintype == "audio":
        fp = open(attach_name, "rb")
        attachment = MIMEAudio(fp.read(), _subtype=subtype)
        fp.close()
    else:
        fp = open(attach_name, "rb")
        attachment = MIMEBase(maintype, subtype)
        attachment.set_payload(fp.read())
        fp.close()
        encoders.encode_base64(attachment)
    attachment.add_header("Content-Disposition", "attachment", filename=attach_name)
    msg.attach(attachment)

    server = smtplib.SMTP("smtp.gmail.com:587")
    server.starttls()
    server.login(username,password)
    server.sendmail(emailfrom, emailto, msg.as_string())
    server.quit()
    print("Send Gmail done")

def crawl_stock_id(name):  
    #catch payload 
    res = requests.post('http://www.tse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_d.php#')
    res.encoding = 'big5'

    #find key & Encryption 
    soup = BeautifulSoup(res.text, 'html.parser') 

    for inp in soup.select('input'):
        if 'hidden' in inp.get('type'): 
            payload[inp.get('name')] = base64.b64encode(inp.get('value').encode('utf-8')) 
        
    #download csw file        
    res2 = requests.post('http://www.tse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_print.php?language=ch&save=csv',data = payload ,stream = True)

    #save to twse.csv 
    f = open(name,'wb')
    copyfileobj (res2.raw,f)
    f.close()  
    
    
def crawl(target):  
    
    
    #catch payload 
    res = requests.post('http://www.tse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_d.php#')
    res.encoding = 'big5'

    #find key & Encryption 
    soup = BeautifulSoup(res.text, 'html.parser') 

    #get & translate date 
    dt = soup.select('input')[2].get('value')
    dt = dt.replace(dt[0:3], str(int(dt[0:3])+ 1911))
    dt = dt.replace(dt[4:5],'-')
    dt_str = dt.split(' ')

    for inp in soup.select('input'):
        if 'hidden' in inp.get('type'): 
            payload[inp.get('name')] = base64.b64encode(inp.get('value').encode('utf-8')) 
        
    #download csw file        
    res2 = requests.post('http://www.tse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_print.php?language=ch&save=csv',data = payload ,stream = True)

    #save to twse.csv 
    f = open('twse.csv','wb')
    copyfileobj (res2.raw,f)
    f.close()  

    #read csw to pandas 
    twse = pandas.read_csv('twse.csv',encoding = "big5",header=1)

    with lite.connect(target) as conn:   
        for i in range(0,max_stock_id):
            col = twse.iloc[i]
            col = col.tolist()
            stock_id = col[0]
            if len(stock_id)>10 :
                break
            else:    
                data = dt_str + col
                conn.execute('INSERT INTO 指標 (日期,代號,名稱,本益比,殖利率,股價淨值比) VALUES (?,?,?,?,?,?);', data)
    conn.close()
    
    print("crawl 指標 is Done")
    
def update_stock_id(name):    

    crawl_stock_id('temp.csv')
    #read csw to pandas 
    twse = pandas.read_csv('temp.csv',encoding = "big5",header=1)
    
    conn = conn_mysql()
    cursor = conn.cursor()    
    for i in range(0,max_stock_id):
        col = twse.iloc[i]
        col = col.tolist()
        del col[2:5]   
        stock_id = col[0]
        stock_name = col[1]
        if len(stock_id)>10:
            break
        else:   
            cursor.execute ("UPDATE 股票代號 SET 代號='%s',名稱='%s' WHERE ID='%d'" % (stock_id,stock_name,i+1))
           
    conn.commit()
    conn.close()  
    
    os.remove("temp.csv")
    print("Stock ID update success !")
    
def create_sql_lib(name):    
    #create sqlite lib
    conn = conn_mysql()
    cursor = conn.cursor()
    cursor.execute("""CREATE TABLE 股票代號 (
                                            ID   INT(12) NOT NULL AUTO_INCREMENT PRIMARY KEY,
                                            代號 VARCHAR(10)CHARACTER SET utf8 COLLATE utf8_general_ci,
                                            名稱 VARCHAR(10)CHARACTER SET utf8 COLLATE utf8_general_ci )""")
    cursor.execute("""CREATE TABLE 股價 (
                                        ID   INT(12) NOT NULL AUTO_INCREMENT PRIMARY KEY,
                                        日期 DATE,
                                        代號 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        名稱 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        開盤價 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        收盤價 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        漲跌 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        漲跌百分比 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        最高 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        最低 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        日量 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        均線50 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        均線200 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        本益比 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        殖利率 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        股價淨值比 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci)""")          

    cursor.execute("""CREATE TABLE 指標 (
                                         ID   INT(12) NOT NULL AUTO_INCREMENT PRIMARY KEY,       
                                         日期 DATE ,
                                         代號 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                         名稱 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci, 
                                         本益比 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                         殖利率 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                         股價淨值比 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci)""")    
    
    cursor.execute("""CREATE TABLE 營收 (
                                         ID   INT(12) NOT NULL AUTO_INCREMENT PRIMARY KEY,       
                                        日期 DATE,
                                        代號 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        名稱 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        EPS VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        月營收 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        季營收 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        年營收 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        配息 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci,
                                        配股 VARCHAR(8) CHARACTER SET utf8 COLLATE utf8_general_ci)""")
    
    print ("MySQL LIB created successfully")  
    
    
    crawl_stock_id('temp.csv')
    
    #read csw to pandas 
    twse = pandas.read_csv('temp.csv',encoding = "big5",header=1)

    #update to sqlite file 
    for i in range(0,max_stock_id):
        col = twse.iloc[i]
        col = col.tolist()
        del col[2:5]
        if len(col[0]) >10:
            break
        else:    
            cursor.execute("INSERT INTO 股票代號 (代號,名稱) VALUES ('%s','%s')"%(col[0],col[1]))   
    conn.commit()           
    conn.close()      

    
    os.remove("temp.csv")
    print ("Stock ID created successfully") 
    
    
def getStock_history(id,start,end):
    stock = Share(str(id)+'.TW')
    data = stock.get_historical(start, end)
    return data


def crawl_history(target):     
    today = date.today() #todays date 
    conn = conn_mysql()
    cursor = conn.cursor(buffered = True)
    cursor.execute("SELECT * FROM 股票代號") 
    for i in range(0,max_stock_id):             
        sql = cursor.fetchone()
        print(sql)
        if sql:    
            try:
                print(sql[2])
                stock = getStock_history(sql[1],'2008-01-01',str(today))    
                for i in range(0,len(stock)):                            
                    data = stock[i]
                    if data['Date']:
                        stock_date = data['Date']
                    if data['Open']:  
                        stock_open = data['Open']
                    if data['High']:                         
                        stock_high = data['High']
                    if data['Low']:                         
                        stock_low = data['Low']
                    if data['Close']:         
                        stock_close = data['Close']
                    if data['Volume']:
                        stock_volume = (int(data['Volume'])/1000)
                    if int(stock_volume):      
                        #sql_list = [stock_date]+[sql[0]]+[sql[1]]+[stock_open]+[stock_close]+[stock_high]+[stock_low]+[stock_volume]
                        cursor2 = conn.cursor(buffered = True)
                        cursor2.execute("INSERT INTO 股價 (日期,代號,名稱,開盤價,收盤價,最高,最低,日量) VALUES ('%s','%s','%s','%s','%s','%s','%s','%d')"%(stock_date,sql[1],sql[2],stock_open,stock_high,stock_low,stock_close,stock_volume))                     
                conn.commit()    
                print(datetime.now())  
            except:
                print("Not support ID =",sql[0])            
    conn.close()    
    print("Crawl history is done")
                        
#main   
def main(fucntion):
        
#    if fucntion == 'create_lib':
#    create_sql_lib('stock.sqlite')      
        
#    elif fucntion == 'update_stock_id':conn.commit(
#    update_stock_id('stock.sqlite')  
    
#    elif fucntion == 'crawl':      
#        crawl('stock.sqlite')  #target sql file name , para    
    stock = crawl_history('stock.sqlite')  
#    elif fucntion == 'crawl_history':      
#        stock = crawl_history('stock.sqlite')   
#    elif fucntion == 'report':
#        send_to_gmail('twse.csv','daily.xlsx') #source pandas structure , target excel file name        
#    else:
#        print(' Usag   hamu create_lib          #建立資料庫')
#        print('        hamu update_stock_id     #更新股票代號')
#        print('        hamu crawl               #爬當日資料')
#        print('        hamu crawl               #爬歷史資料')
#        print('        hamu report              #透過gmail寄出電子報')
        
        
if __name__ == '__main__':
    main(sys.argv[1])     

Connector to MYSQL Server: 192.168.0.110 SUCCESS
(1, '1101', '台泥')
台泥
2017-03-23 23:56:06.150628
(2, '1102', '亞泥')
亞泥
2017-03-23 23:56:50.675853
(3, '1103', '嘉泥')
嘉泥
2017-03-23 23:58:08.680023
(4, '1104', '環泥')
環泥
2017-03-23 23:58:32.955625
(5, '1108', '幸福')
幸福
2017-03-23 23:58:51.132701
(6, '1109', '信大')
信大
2017-03-23 23:59:11.331102
(7, '1110', '東泥')
東泥
2017-03-23 23:59:27.846971
(8, '1201', '味全')
味全
2017-03-24 00:00:00.222620
(9, '1203', '味王')
味王
2017-03-24 00:00:17.783183
(10, '1210', '大成')
大成
2017-03-24 00:00:42.031238
(11, '1213', '大飲')
大飲
2017-03-24 00:00:59.043137
(12, '1215', '卜蜂')
卜蜂
2017-03-24 00:01:39.003635
(13, '1216', '統一')
統一
2017-03-24 00:02:32.323394
(14, '1217', '愛之味')
愛之味
2017-03-24 00:02:49.148109
(15, '1218', '泰山')
泰山
2017-03-24 00:03:07.976959
(16, '1219', '福壽')
福壽
Not support ID = 16
(17, '1220', '台榮')
台榮
2017-03-24 00:03:36.789756
(18, '1225', '福懋油')
福懋油
2017-03-24 00:03:57.378716
(19, '1227', '佳格')
佳格
Not support ID = 19
(20, '1229', '聯華')
聯華
2017-03-24 00:04: